[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset

Let's first of all have a look at the data. You can download the file `labeledTrainData.tsv` on the [Kaggle website of the competition](https://www.kaggle.com/c/word2vec-nlp-tutorial/data), or on our [Google Drive](https://drive.google.com/file/d/1a1Lyn7ihikk3klAX26fgO3YsGdWHWoK5/view?usp=sharing)


In [56]:
# Import pandas, numpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [57]:
# Read dataset with extra params sep='\t', encoding="latin-1"
df = pd.read_csv('C:/Users/Admin/Downloads/Week 5 - Logistic Regression/labeledTrainData.tsv',sep='\t', encoding="latin-1")
df.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [58]:
df = df.drop(['id'], axis=1)
df.head()

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


## 2. Preprocessing

In [59]:
# stop words
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = stopwords.words('english')

from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [60]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = BeautifulSoup(text, "lxml").get_text()
    text = re.sub('<[^<]*>', '', text) # Your code here
    text = re.sub(r'\d+',' ', text) #Remove all digits
    text = re.sub(r'\s+[a-z]\s+',' ', text) #Remove all single letters
    text = re.sub(r'^\s+','', text) #Remove space from start of text
    text = re.sub(r'\s+$','', text) #Remove space from end of text
    text = re.sub(r'\s+',' ', text) #Remove all multi space    
    
    # Save emoticons for later appending
    emoticons_pattern = re.compile('(?::|;|=)(?:-)?(?:\)|\(|D|P)')
    emoticons = [ i.group() for i in emoticons_pattern.finditer(text)]
    
    text = emoticons_pattern.sub('--', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case

    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    return text

In [61]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [62]:
# split the dataset in train and test
# Your code here
from sklearn.model_selection import train_test_split

X = df['review']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [63]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)
clf = Pipeline([
                ('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))
                ])
clf.fit(X_train, y_train)

F:\Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Pipeline(steps=[('vect',
                 TfidfVectorizer(preprocessor=<function preprocessor at 0x000001D71FA3C670>,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...],
                                 tokenizer=<function tokenizer_porter at 0x000001D71FA3CB80>)),
                ('clf', LogisticRegression(random_state=0))])

## 4. Evaluate Model

In [64]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

pred = clf.predict(X_test)
print('Accuracy:',accuracy_score(y_test,pred))
print('Confusion matrix:\n',confusion_matrix(y_test,pred))
print('Classification report:\n',classification_report(y_test,pred))

Accuracy: 0.8889333333333334
Confusion matrix:
 [[3311  470]
 [ 363 3356]]
Classification report:
               precision    recall  f1-score   support

           0       0.90      0.88      0.89      3781
           1       0.88      0.90      0.89      3719

    accuracy                           0.89      7500
   macro avg       0.89      0.89      0.89      7500
weighted avg       0.89      0.89      0.89      7500



## 5. Export Model 

In [65]:
# Using pickle to export our trained model
import pickle
import os

In [66]:
pickle.dump(clf, open(os.path.join('C:/Users/Admin/Downloads/Week 5 - Logistic Regression', 'SentimentAnalysis.pkl'), 'wb'), protocol=4)